# TP sur la résolution des systèmes linéaires (n équations et n inconnues)

In [2]:
import numpy as np

## I)Résolution de Ax=b avec A une matrice carrée nxn et b ∈ Rn

### 1) Résolution d'un système linéaire triangulaire

#### a) supérieur

In [3]:
def solve_upper_triangular(A, b):
    n = len(A)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        sum_ax = sum(A[i][j] * x[j] for j in range(i+1, n))
        x[i] = (b[i] - sum_ax) / A[i][i]
    return x

#### b) inférieur

In [4]:
def solve_lower_triangular(A, b):
    n = len(A)
    x = np.zeros(n)
    for i in range(n):
        sum_ax = sum(A[i][j] * x[j] for j in range(i))
        x[i] = (b[i] - sum_ax) / A[i][i]
    return x

### 2)Méthode de Gauss

#### a) Méthode de triangulation de Gauss

In [5]:
def gauss_elimination(A, b):
    n = len(A)
    for i in range(n):
        factor = A[i][i]
        A[i] = [a_ij / factor for a_ij in A[i]]
        b[i] /= factor
        for k in range(i+1, n):
            factor = A[k][i]
            A[k] = [a_kj - factor * a_ij for a_kj, a_ij in zip(A[k], A[i])]
            b[k] -= factor * b[i]
            
    x = [0 for _ in range(n)]
    for i in range(n-1, -1, -1):
        x[i] = b[i] - sum(A[i][j] * x[j] for j in range(i+1, n))
    
    return x

#### b) Appel à la résolution

In [6]:
A_new = np.random.rand(5, 5)
b_new = np.random.rand(5)

x_solution = gauss_elimination(A_new.copy(), b_new.copy())
x_solution

[-0.08240913109592363,
 1.936861998932319,
 -3.4638173205021574,
 2.0235370632272875,
 -0.7363150504803768]

### 3) Résolution par décomposition LU

#### a) Décomposition LU de la matrice A

In [7]:
from scipy.linalg import lu
import numpy as np

def lu_decomposition(A):
    P, L, U = lu(A)
    return P, L, U

A_for_LU = np.random.rand(5, 5)
P_LU, L_LU, U_LU = lu_decomposition(A_for_LU)

#### b) Résolution du système initial par la résolution de deux systèmes triangulaires (supérieur et inférieur)

In [8]:
def solve_lu(P, L, U, b):
    
    b = P @ b
    
    y = np.zeros_like(b)
    for i in range(len(b)):
        y[i] = b[i] - L[i,:i].dot(y[:i])
    
 
    x = np.zeros_like(y)
    for i in range(len(b)-1, -1, -1):
        x[i] = (y[i] - U[i,i+1:].dot(x[i+1:])) / U[i,i]
        
    return x

### 4) Méthodes itératives
Si vous avez le temps, vous écrirez les méthodes par calcul des éléments de xi et également l'écriture matricielle.

#### a) Méthode de Jabobi

In [9]:
import numpy as np

def jacobi(A, b, iterations=25, p=1e-10):
    n = len(A)
    x = np.zeros_like(b)
    for _ in range(iterations):
        x_new = np.array([(b[i] - sum(A[i][j] * x[j] for j in range(n) if j != i)) / A[i][i] for i in range(n)])
        if np.linalg.norm(x_new - x, np.inf) < p:
            break
        x = x_new
    return x



#### b) Méthode de Gauss-Seidel

In [10]:
def gauss_seidel(A, b, iterations=25, p=1e-10):
    n = len(A)
    x = np.zeros_like(b)
    for _ in range(iterations):
        for i in range(n):
            x[i] = (b[i] - sum(A[i][j] * x[j] for j in range(i)) - sum(A[i][j] * x[j] for j in range(i + 1, n))) / A[i][i]
        if all(abs(A[i].dot(x) - b[i]) < p for i in range(n)):
            break
    return x

In [11]:
A = np.array([[4, -1, 0, 0], [-1, 4, -1, 0], [0, -1, 4, -1], [0, 0, -1, 3]])
b = np.array([15, 10, 10, 10])

x_jacobi = jacobi(A, b)
x_gauss_seidel = gauss_seidel(A, b)

print("Jacobi:", x_jacobi)
print("Gauss-Seidel:", x_gauss_seidel)

Jacobi: [5. 5. 5. 5.]
Gauss-Seidel: [4 4 4 4]


#### c) Méthode de relaxation

In [12]:
def relaxation(A, b, x0=None, alpha=3**(-1), maxiter=5, precision=10**-3):
    assert alpha < 1
    iters = 0
    xk = x0.copy()
    n = A.shape[0]
    print(x0)
    while np.linalg.norm(b-A@xk) > precision:
        xkp=xk.copy()
        for i in range(n): 
            xkm =xk.copy()
            xkm[i] = 0
            xkm[:i] = xkp[:i]
            xkp[i] = alpha* (b[i] - A[i, :]@xkm)/A[i, i] + (1-alpha)*xk[i]
        xk = xkp

        print(xk)
        iters += 1
        if iters == maxiter:
            break
        
        
    return xk

relaxation(A, b, np.zeros((4,)), maxiter=5)

[0. 0. 0. 0.]
[1.25       0.9375     0.91145833 1.21238426]
[2.16145833 1.71440972 1.68487172 2.10657525]
[2.8338397  2.35283243 2.32819845 2.77418333]
[3.33529584 2.87384615 2.85613476 3.27791497]
[3.71301774 3.29666014 3.28530443 3.66142158]


array([3.71301774, 3.29666014, 3.28530443, 3.66142158])

## II)Implantez la méthode du calcul de l'inverse d'une matrice carrée A par "double triangulation", puis division des lignes par les éléments diagonaux.